In [ ]:
# import packages 
import cv2
import numpy as np
import os
import datetime
import time

%matplotlib inline

# folder path and name, define your own paths here
project_path = os.getcwd()
video_folder = os.path.join(project_path,'input_videos') # input video folder
image_folder = os.path.join(project_path,'snapshot_highfreq') # output image frames folder
image_name_format = '%Y%m%d%H%M%S'

# Operating parameter
# Specify the frequency you want to capture the frames
capture_interval = datetime.timedelta(seconds=60)  # in s

# Specify the start and end date
start_date = datetime.datetime(2018,10,2)
end_date =  datetime.datetime(2018,10,3)

In [ ]:
def crop_central_image(img, x_length, y_length):
    image_shape = np.array(img.shape[0:2])
    cropped_shape = np.array([x_length, y_length])
    x_low, y_low = ((image_shape - cropped_shape) / 2).astype('int')
    x_high, y_high = ((image_shape + cropped_shape) / 2).astype('int')
    img = img[x_low:x_high, y_low:y_high, :]
    return img

def listdir_noini(path):
    f_list = []
    for f in os.listdir(path):
        _, file_extension = os.path.splitext(f)
        if not file_extension == '.ini':
            f_list.append(f)
            
    return sorted(f_list)

def parse_video_name(video_name):
    # ignore the file if it's an image file
    _,extension = os.path.splitext(video_name)
    if (extension == '.jpg') or (extension == '.db'):
        return None, False
    
    # Interpret video file name into time stamp
    if len(video_name) < 30:
        video_name_format = "%Y%m%d-%H%M%S-01.avi"
    else:
        video_name = video_name.split('_')[-3]
        video_name_format = "%Y%m%d%H%M%S"
    
    # obtaining initial time
    try:
        initial_time = datetime.datetime.strptime(video_name, video_name_format)
    except:# there are some other files in the folder
        print("can't handle ", video_name)
        return None, False

    return initial_time, True

def get_dest_dir(img_time):
    year_name = '{:04d}'.format(img_time.year)
    month_name = '{:02d}'.format(img_time.month)
    day_name = '{:02d}'.format(img_time.day)
    return os.path.join(year_name,month_name,day_name)

In [ ]:
############### if start data is provided, please skip this cell!#############

# if start date is not provided find start date of the current capture process
snapped_list = []

years_list = listdir_noini(os.path.join(image_folder))
for year in years_list: # cycle through the years
    months_list = listdir_noini(os.path.join(image_folder,year))
    for month in months_list: # cycle through the months
        dates_list  = listdir_noini(os.path.join(image_folder,year,month))
        for date in dates_list: # cycle through each day
            snapped_list += listdir_noini(os.path.join(image_folder,year,month,date))
        
start_date = datetime.datetime.strptime(sorted(snapped_list)[-1], image_name_format+'.jpg')
print('Last time, snapshot is finished for: ',start_date)

Last time, snapshot is finished for:  2018-10-03 17:45:00


In [ ]:
# Snapshot from video at ten second interval
# Initialization: time and frame counter
tic = time.process_time()
frame_cnt = 0

# Step through every single video, but only open the streams that are relevant
# list all years  
years_list = listdir_noini(os.path.join(video_folder))
for year in years_list: # cycle through the years
    months_list = listdir_noini(os.path.join(video_folder,year))
    for month in months_list: # cycle through the months
        dates_list  = listdir_noini(os.path.join(video_folder,year,month))
        for date in dates_list: # cycle through each day
            
            # obatain all files in the current folder
            curr_folder = os.path.join(video_folder,year,month,date)
            video_filenames = listdir_noini(curr_folder)
            print('processing ',month,'-',date)
            # only process files in the defined date range
            curr_date = datetime.datetime.strptime(date,'%Y%m%d')
            if (curr_date<start_date) or (curr_date>end_date):
                continue          
            for video_name in video_filenames: # cycle through each file of the day
                # Attempt to parse the filename
                initial_time,is_relevant = parse_video_name(video_name)
                # if the file is deemed irrelevant, move on to the next
                if is_relevant == False:
                    continue
                
                print('Capturing images from video stream:',video_name)
                # find end_time of this video
                end_time = datetime.datetime.strptime(video_name.split('_')[-2],image_name_format)
                
                # shift to the first n*10s 
                shift = datetime.timedelta(seconds= 10 + initial_time.second//10*10 - initial_time.second)  # in s
                video_pos_time = shift.total_seconds() * 1000 # in ms
                curr_time = initial_time + shift
                
                # open video stream
                cap = cv2.VideoCapture(os.path.join(curr_folder,video_name))

                while cap.isOpened():
                    cap.set(0, video_pos_time)
                    ret, frame = cap.read()
                    
                    if ret and (curr_time <= end_time):
                        # prompt progress
                        if curr_time.minute%10 == 0 and curr_time.second == 0:
                            print('processing ',curr_time.strftime('%H%M%S'))
                        
                        # crop and save the central square
                        frame = crop_central_image(frame,frame.shape[0],frame.shape[0])
                        frame_filename = curr_time.strftime(image_name_format) + ".jpg"
                        
                        # Create image folder if not exist yet
                        dest_dir = get_dest_dir(curr_time)
                        if not os.path.exists(os.path.join(image_folder, dest_dir)):
                            os.makedirs(os.path.join(image_folder, dest_dir))
                        cv2.imwrite(os.path.join(image_folder, dest_dir, frame_filename), frame)

                        #Move forward in time    
                        video_pos_time += capture_interval.total_seconds() * 1000  # in ms
                        curr_time += capture_interval
                    else :
                        break
                cap.release()
            print('time_elapsed',time.process_time()-tic)


processing  10 - 20181002
processing  10 - 20181003
